# Final project

## Project title

Research Question: Which UvA courses do I meet the entry requirements for? XXX






## Process Book

1. Don’t forget the process book. Maintain it every day you’ve worked on the project. This is the main way for 
   us to see what you’ve learned and is an important part of the evaluation.
2. Don’t use it for everything. Don’t spend too much time on your process book! Ten minutes 
   every day you’ve worked on the project on this, should be more than enough.
3. Keep it simple and short. Use simple language, short sentences, and be as straightforward as possible when 
   documenting your learning process.
4. Create a simple template. In order to organize your process book, it is nice to create and use a template:


### Date of entry: 25. Jan
#### What I’ve worked on: 
Git
- Lecture git
- Tried out git by myself
- Created git repository

Project choice
- Learn to use tableau and uber data as I am applying to intern at uber and tableau is a desired skill

Tableau
- Downloaded tableau 
- See what I can use tableau for to decide which project I want to do specifically
- Learn tableau basics
- Experiment with DP datasets (e.g., weather data) in tableau
- Experiment with uber selection dataset --> too easy (?)
    
#### What problems I encountered:
- Cannot use dataset I first thought about using

#### What I learned:
- How to use git
- Tableau basics https://www.youtube.com/watch?v=jEgVto5QME8
- I want to gather my own uber data e.g., through web-scraping, or API

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide, uber data XXX




### Date of entry: 26. Jan
#### What I’ve worked on: 
Tableau
- Learned more functions of tableau and experimented with these  

Project
- Created jupyter notebook
- Started process book

Brainstorm
- Potentially make website with tableau dashboard
- Which data can I use? 

Find data
- Look for uber APIs

#### What problems I encountered: 
- Uber APIs are not open, I can not use them 

#### What I learned: 
Tableau
- Basic functions: https://www.youtube.com/watch?v=yejulE4b-3Y
- Use and join (csv) files: https://www.youtube.com/watch?v=5WXrJ1JvTOo
- Integrate tableau dashboard in website: https://www.youtube.com/watch?v=wJ2CHIJalNU

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide



### Date of entry: 27. Jan
#### What I’ve worked on: 
- Find new project ideas
- Decided for final topic/Question
- Changed strategy for scraping data: work with individual course pages instead of course overview

#### What problems I encountered: 
- Url of studiegids does not change per page of courses

#### What I learned: 

#### Which resources did I use:
- Multiple forms to try to solve the url problem, but didn't find a solution




### Date of entry: 30. Jan
#### What I’ve worked on: 
- Scraping course data

#### What problems I encountered: 
- I cannot simply check whether a unique number-combination in the end of the link opens a specific course page by testing if the link is dead or not: if the number in the end doesn't link to a course it still opens the studigids page


#### What I learned: 
- Scrape to dictionary XXX
- Scrape with slideout website XXX

#### Which resources did I use:

## To-Do

1. Scrape / Transform data 
2. Import data into tableau
3. (Webpage with tableau dashboard)

## 1. Data

Relevant personal data

Programme: ""
Level: Bachelor's, Master's (Master's, Master), Exchange programme ('Exchange Programme Exchange Programme UvA', 'Exchange Programme Exchange Programme'), pre-Master's programme, Secondary Subject/Bijvak('?'), 'Honours MSc', 'Research Master's
Year of the studies: ""
Credits obtained this/1st year: ""

url: https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course


Relevant course information 

Course name: Course name
Entry requirements: Course > Entry requirements > Split by sentence & look for keywords (if not present, add to dict with unknown entry requirements)
Level: Course > Is part of > loop through Level list (s.a.) 
Field: Course > Is part of > remove Level > everything left 

(Coordinator name: Course > Lecturer(s) > Coordinator ('co-ordinator')
(Coordinator email: Course > Lecturer(s) > Coordinator (crawl) > Email)


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import numpy as np
from math import ceil
import sys

In [2]:
#Prepare webscraping with BeautifulSoup

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
        }
        with closing(get(url, stream=True, headers=headers)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        print('The following error occurred during HTTP GET request to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

- All courses have individual descriptions with unique urls
- First, I will find all urls that link to courses by trying out urls
- I can check whether I found all courses by comparing the length of my url list to the number or courses stated on the website
- After that, I will loop through the urls that work and scrape the data for all courses

In [14]:
#SCRAPER: name, urls, levels and fields (subset of the data) 

potential_levels = ["Bachelor's", "Dual Master's", "Master's", "Exchange Programme Exchange Programme UvA", 
                    "Exchange programme Exchange Programme", "Exchange programme Exchange Programme UvA", 
                    "Exchange Programme Exchange Programme", "pre-Master's programme", "Secondary Subject", 
                    "Bijvak", "Honours MSc", "Research Master's", "Minor"] #tailored to english website

courses = {}
unique_n = 102590
    
while unique_n < 102600:
    url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/{unique_n}'
    html = simple_get(url)
    dom = BeautifulSoup(html, 'html.parser')
    
    unique_n += 1
    
    if (dom.find('article', class_ = 'twelve columns')) is None: #no course under this url - go to next url
        pass
    
    else: #course exists
        name = dom.find('article', class_ = 'twelve columns').h1.text[1::]
        courses[name] = {}
        #print(name)
        courses[name]['url'] = url
        courses[name]['Academic level'] = [] #Add empty list in case there is no data
        courses[name]['Field of study'] = [] #Add empty list in case there is no data
        courses[name]['Entry requirements'] = [] #Add empty list in case there is no data

        #print(url)
        
        for info_block in dom.find_all('tr', {'class':''}):
            info_block = info_block.text


            #Education level and field of the course
            if 'Is part of' in info_block: #Find text box that contains level and field of the course
                is_part_of = info_block.replace("Is part of", "") #Remove title of the text box

                # Education level of the course e.g., Bachelor's
                levels = [] 
                for l in potential_levels:
                    if l in is_part_of:
                        levels.append(l)

                # Field(s) of the course e.g., Business
                for lev in levels: 
                    is_part_of = is_part_of.replace(lev+" ", ";") #replace education level by semicolon to split on the semicolon later
                fields = is_part_of[1::].split(";") #remove first semicolon to not get an empty element in the study fields list 
                                                    #and split on semicolon to get this list
        
                courses[name]['Academic level'] = levels
                courses[name]['Field of study'] = fields

        
        #entry = 

print(courses)

{'Work, Power & Income:\xa0Why regulation fails to address labour market inequality (&sometimes succeeds)': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102591', 'Academic level': ["Bachelor's"], 'Field of study': ['Politics, Psychology, Law and Economics (PPLE)'], 'Entry requirements': []}, 'American Think Tanks and U.S. Foreign Policy ': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102593', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}, '"Losing Earth"?: Activism and Diplomacy on the Environment and Climate since 1968': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}}


In [3]:
# SCRAPER WORKSPACE: make entry requirements categorical

#separators = . and , 

#indicators = 'priority', 'ec', 'first year', 'research participation', 'only', 'selected into', 
#              'completed', 'second year',     course ids, study programme names

url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/101937'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')


In [7]:
# SCRAPER WORKSPACE 

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text
    
    # Entry requirements of the course: full text
    if 'Entry requirements' in info_block: #Find text box with entry requirements
        entry = info_block.replace("Entry requirements", "") #Remove title of the text box
        entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining
        #entry = entry.lower()
        
        
        #Split into sentences as these contain seperate criteria
        entry = entry.replace(". ", ".").split(".")
        entry = list(filter(None, entry))
        #print(entry)
        #remove filler words XXX
        
        
        cred_l = ['ec', 'ects', 'ect', 'credits', 'EC', 'ECTs', 'Credits']
        credits = {}
        
        for sentence in entry:
            ### HARD REQUIREMENTS
            
            # CREDIT REQUIREMENTS   
            words = sentence.split()
            if 'ec' or 'ects' or 'ect' or 'credits' or 'EC' or 'ECTs' or 'Credits' in sentence:

                for i in range(len(words)):
                    if words[i] in cred_l: #find position of 'ec'
                        
                        n_ecs = words[i-1] #Find the number of credits (element before 'ec')
                        #print(f'Position of ec {i}')
                        
                        co = 1
                        x = 1
                        
                        while x == 1:
                            next_w = words[i+co]
                            #print(next_w)
                            
                        #credits required from first year XXX add second year
                            first_l = ['first-year', 'first year']
                            if next_w in first_l:# or 'first year':
                                credits['First year'] = n_ecs # XXX credits[f'First year {'the_study_programme'}']
                                x = 0
                                
                        #credits required from a course: find course name
                            #check whether course name consists of multiple words by checking capitalisation
                            elif next_w[0] in upper:
                                co2 = 1

                                while words[i+co+co2][0] in upper:
                                    next_w += " " + words[i+co+co2]
                                    co2 += 1
                                credits[next_w] = n_ecs
                                x = 0
                            
                            #one word course name
                            else:
                                co+=1
                                #print('no')
                
            # COURSE REQUIREMENTS (having completed specific courses)
                #print(ecs)

                
            ### SOFT REQUIREMENTS    
            # PRIORISATIONS
                
print(credits)
#courses[name]['Entry requirements'] = credits


{'First year': '60', 'Clinical Psychology': '18'}


In [57]:

# Extract list of course titles from the entry requirements text; based on capitalisation; sensitive to words
# that are typically not capitalised in titles (conjunctions, articles, prepositions)

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
sentence = 'Scientific and Statistical Reasoning (7202A701XY), Practical Training: Psychological Research (7202A704), ≥ 18 ec of the course package Clinical Psychology and the first part of Evidence-based Clinical Practice: Research Methods/CP (72037001).'
words = sentence.split()
print(words)



def find_c_name(words): #input: list of words of one sentence from the entry requirements text
    
    # List of words not capitalised in headings
    conjunctions = ["and", "or", "but", "nor", "yet", "so", "for"]
    articles = ['a', 'an', 'the']
    prepositions = ["in", "to", "of", "at", "by", "up", "for", "off", "on"]
    no_cap = conjunctions + articles + prepositions

    
    
    string = ''
    
    # Find wordgroups of nouns and conjunctions/articles/prepositions
    for i in range(len(words)):
        if words[i][0] in upper or words[i] in no_cap:
            string += words[i] + " "
            #print(string)
        else:
            string = string[:-1]
            string += "=" 
    string = string[:-1] #remove last split sign (=)
    c_names_raw = string.split('=')
    #print(c_names_raw)
    
    c_names = []
    
    
    
    # Delete non-capitalised words that embedded in capitalised words - these are likely not part of the course title (e.g., 'of the', 'Clinical Psychology and the', 'of Evidence-based Clinical Practice: Research Methods/CP')
    for ele in c_names_raw:
        words = ele.split()
        
        while len(words) != 0 and words[0][0] not in upper:
            del words[0]
        
        while len(words) != 0 and words[-1][0] not in upper:
            del words[-1]
            
        ele = ' '.join(words)
        
        if len(ele) != 0: #do not append empty course names
            c_names.append(ele)
        #print(ele)

            
    return(c_names)


print(find_c_name(words))

        

['Scientific', 'and', 'Statistical', 'Reasoning', '(7202A701XY),', 'Practical', 'Training:', 'Psychological', 'Research', '(7202A704),', '≥', '18', 'ec', 'of', 'the', 'course', 'package', 'Clinical', 'Psychology', 'and', 'the', 'first', 'part', 'of', 'Evidence-based', 'Clinical', 'Practice:', 'Research', 'Methods/CP', '(72037001).']
['Scientific and Statistical Reasoning', 'Practical Training: Psychological Research', 'Clinical Psychology', 'Evidence-based Clinical Practice: Research Methods/CP']


In [2]:


url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text

    # Coordinator information (optional)
    if 'Lecturer(s)' in info_block:
        #print(info_block) #coord, coord_mail

        
        
#def extract_courses: 



#Special cases

https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595 #(MA) or (BA)
https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94517 #Dual Master's / Master's

IndentationError: expected an indented block (568588304.py, line 14)